In [5]:
!pip  install transformers datasets rouge-score nltk


In [6]:
# Import necessary libraries
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset, load_metric
import nltk

# Download required NLTK data
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pujit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# Load the PEGASUS model and tokenizer
model_name = 'google/pegasus-cnn_dailymail'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Load the CNN/DailyMail dataset
dataset = load_dataset('cnn_dailymail', '3.0.0', split='test[:1%]')  # Use a small subset for quick evaluation


In [15]:
# Initialize ROUGE and METEOR metrics
rouge = load_metric('rouge')
meteor = load_metric('meteor')

C:\Users\pujit\AppData\Local\Temp\ipykernel_18384\2828924599.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric('rouge')
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pujit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pujit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\pujit\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [16]:
# Function to generate summary and evaluate
def evaluate_pegasus(example):
    inputs = tokenizer(example['article'], return_tensors='pt', truncation=True, padding='longest')
    summary_ids = model.generate(inputs['input_ids'])
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return {'reference': example['highlights'], 'summary': summary}

# Generate summaries and evaluate metrics
results = dataset.map(evaluate_pegasus, batched=False)


Map:   0%|          | 0/115 [00:00<?, ? examples/s]

In [17]:
rouge_result = rouge.compute(predictions=results['summary'], references=results['reference'], use_stemmer=True)
rouge_result = {key: value.mid.fmeasure for key, value in rouge_result.items()}

In [18]:
meteor_result = meteor.compute(predictions=results['summary'], references=results['reference'])


In [19]:
# Print the results
print("ROUGE Scores:")
print(f"ROUGE-1: {rouge_result['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_result['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_result['rougeL']:.4f}")

ROUGE Scores:
ROUGE-1: 0.3305
ROUGE-2: 0.1288
ROUGE-L: 0.2445


In [21]:
print("\nMETEOR Score:")
print(f"METEOR: {meteor_result['meteor']:.4f}")


METEOR Score:
METEOR: 0.2575
